## Setup and Import Libraries

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Chroma

Chroma is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

### Text Splitting

In [3]:
loader = TextLoader(file_path='speech.txt')
data = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0
)

In [6]:
docs = text_splitter.split_documents(data)
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_

### Embeddings and Vector Store

In [7]:
embeddings = (
    OllamaEmbeddings(model='gemma:2b')  ## By Default = llama2
)

In [8]:
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings
)
vector_db

### Querying Vector Database

In [9]:
query1 = "What does the speaker believe is the main reason the United States should enter the war?"
query2 = "How does the speaker describe the desired outcome of the war?"

In [11]:
result = vector_db.similarity_search(query=query1)
print(result[0].page_content)

government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.


In [12]:
result = vector_db.similarity_search(query=query2)
print(result[0].page_content)

democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.


### Retriever

In [13]:
retriever = vector_db.as_retriever()
retriever_results = retriever.invoke(query1)

In [14]:
retriever_results[0].page_content

'government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'

### Saving the Vector Database

In [15]:
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

### Loading the Vector Database

In [16]:
new_db = Chroma(
    persist_directory="./chroma_db/",
    embedding_function=embeddings
)

In [17]:
result = new_db.similarity_search(query=query1)
result

[Document(id='dc041f9e-05fa-45a2-961f-f7347f96e4a5', metadata={'source': 'speech.txt'}, page_content='government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'),
 Document(id='1ba0f9e1-2f6c-42dd-a24c-0f77fac2cc02', metadata={'source': 'speech.txt'}, page_content='democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 D

In [18]:
result[0].page_content

'government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'

### Similarity Score

In [21]:
result = new_db.similarity_search_with_score(query=query1)
result[0]

(Document(id='dc041f9e-05fa-45a2-961f-f7347f96e4a5', metadata={'source': 'speech.txt'}, page_content='government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'),
 0.5711133092902252)